<a href="https://colab.research.google.com/github/marco-siino/McRock-SemEval-2022-Task4/blob/main/SemEval2022_Task4_SubTask2_Submission2_MSiino_XLNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Main imports and code

In [ ]:
# check which gpu we're using
!nvidia-smi

Mon Jan 31 15:35:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install simpletransformers
!pip install tensorboardx

     |████████████████████████████████| 248 kB 5.5 MB/s 
     |████████████████████████████████| 312 kB 34.1 MB/s 
     |████████████████████████████████| 1.2 MB 32.9 MB/s 
     |████████████████████████████████| 3.5 MB 26.4 MB/s 
     |████████████████████████████████| 6.8 MB 11.4 MB/s 
     |████████████████████████████████| 9.7 MB 35.3 MB/s 
     |████████████████████████████████| 43 kB 1.4 MB/s 
     |████████████████████████████████| 1.7 MB 39.6 MB/s 
     |████████████████████████████████| 895 kB 44.4 MB/s 
     |████████████████████████████████| 596 kB 36.9 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 143 kB 44.1 MB/s 
     |████████████████████████████████| 97 kB 6.3 MB/s 
     |████████████████████████████████| 180 kB 48.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 243 kB 46.5 MB/s 
     |████████████████████████████████| 133 kB 48.1 MB/s 
     |█████████████████

     |████████████████████████████████| 124 kB 5.5 MB/s 


In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs
from urllib import request
import pandas as pd
import logging
import os
import torch

import tensorflow as tf
from collections import Counter
from ast import literal_eval

# Fetch the official scorer

In [ ]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/evaluation.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/evaluation.py


# Fetch all the files needed

In [ ]:
urlDontPatronizeMe_PCL_tsv = "https://drive.google.com/uc?export=download&id=1KAncWruZ4OkKlvEGnxLkNbR1sxAErG8_"
urlDontPatronizeMe_categories_tsv = "https://drive.google.com/uc?export=download&id=1KMZJsskzKLbM-kgYiwXIF0h3Shj4F0mM"
urlTestSet_csv = "https://drive.google.com/uc?export=download&id=161-_6MH16_UHtLTqt0nd09l68pP5MEbQ"
urlDevSet_csv = "https://drive.google.com/uc?export=download&id=1KNuZ_h7NXTSwEz3_0XkaEd4DUAyxojAU"
urlTrainSet_csv = "https://drive.google.com/uc?export=download&id=1KVRrMC9UVwtQE9QfcLv8b11P5t2BhA8I"

tmp = tf.keras.utils.get_file("dontpatronizeme_pcl.tsv", urlDontPatronizeMe_PCL_tsv,
                                    extract=False, cache_dir='.',
                                    cache_subdir='')
tmp = tf.keras.utils.get_file("dontpatronizeme_categories.tsv", urlDontPatronizeMe_categories_tsv,
                                    extract=False, cache_dir='.',
                                    cache_subdir='')
tmp = tf.keras.utils.get_file("task4_test.csv", urlTestSet_csv,
                                    extract=False, cache_dir='.',
                                    cache_subdir='')
tmp = tf.keras.utils.get_file("dev_semeval_parids-labels.csv", urlDevSet_csv,
                                    extract=False, cache_dir='.',
                                    cache_subdir='')
tmp = tf.keras.utils.get_file("train_semeval_parids-labels.csv", urlTrainSet_csv,
                                    extract=False, cache_dir='.',
                                    cache_subdir='')


253952/241839 [===============================] - 0s 0us/step


In [ ]:
# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# check gpu
cuda_available = torch.cuda.is_available()

print('Cuda available? ',cuda_available)

Cuda available?  True


In [ ]:
if cuda_available:
  import tensorflow as tf
  # Get the GPU device name.
  device_name = tf.test.gpu_device_name()
  # The device name should look like the following:
  if device_name == '/device:GPU:0':
      print('Found GPU at: {}'.format(device_name))
  else:
      raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


# Fetch Don't Patronize Me! data manager module

In [ ]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [ ]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

In [ ]:
from dont_patronize_me import DontPatronizeMe

In [ ]:
dpm = DontPatronizeMe('.', '.')

In [ ]:
dpm.load_task1()
dpm.load_task2(return_one_hot=True)

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


# Load paragraph IDs

In [ ]:
trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')

In [ ]:
trids.head()

,par_id,label
0,4341,"[1, 0, 0, 1, 0, 0, 0]"
1,4136,"[0, 1, 0, 0, 0, 0, 0]"
2,10352,"[1, 0, 0, 0, 0, 1, 0]"
3,8279,"[0, 0, 0, 1, 0, 0, 0]"
4,1164,"[1, 0, 0, 1, 1, 1, 0]"


In [ ]:
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)

# Rebuild training set (Task 2)

In [ ]:
rows2 = [] # will contain par_id, label and text
for idx in range(len(trids)):  
  parid = trids.par_id[idx]
  label = trids.label[idx]
  keyword = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].keyword.values[0]
  country = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].country.values[0]
  # select row from original dataset to retrieve the `text` value
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  complete_text = keyword+' '+country+' '+text
  rows2.append({
      'par_id':parid,
      'text':complete_text,
      'label':label
  })
  

In [ ]:
trdf2 = pd.DataFrame(rows2)

In [ ]:
trdf2

,par_id,text,label
0,4341,poor-families gb The scheme saw an estimated 1...,"[1, 0, 0, 1, 0, 0, 0]"
1,4136,homeless za Durban 's homeless communities rec...,"[0, 1, 0, 0, 0, 0, 0]"
2,10352,poor-families lk The next immediate problem th...,"[1, 0, 0, 0, 0, 1, 0]"
3,8279,vulnerable nz Far more important than the impl...,"[0, 0, 0, 1, 0, 0, 0]"
4,1164,poor-families gh To strengthen child-sensitive...,"[1, 0, 0, 1, 1, 1, 0]"
...,...,...,...
8370,8380,refugee gb Rescue teams search for survivors o...,"[0, 0, 0, 0, 0, 0, 0]"
8371,8381,hopeless ke The launch of ' Happy Birthday ' t...,"[0, 0, 0, 0, 0, 0, 0]"
8372,8382,"homeless pk The unrest has left at least 20,00...","[0, 0, 0, 0, 0, 0, 0]"
8373,8383,hopeless pk You have to see it from my perspec...,"[0, 0, 0, 0, 0, 0, 0]"


In [ ]:
trdf2.label = trdf2.label.apply(literal_eval)

# Rebuild test set (Task 2)

In [ ]:
rows2 = [] # will contain par_id, label and text
for idx in range(len(teids)):  
  parid = teids.par_id[idx]
  label = teids.label[idx]
  keyword = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].keyword.values[0]
  country = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].country.values[0] 
  #print(parid)
  # select row from original dataset to access the `text` value
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  complete_text = keyword+' '+country+' '+text
  rows2.append({
      'par_id':parid,
      'text':complete_text,
      'label':label
  })
  

In [ ]:
tedf2 = pd.DataFrame(rows2)

In [ ]:
tedf2

,par_id,text,label
0,4046,hopeless us We also know that they can benefit...,"[1, 0, 0, 1, 0, 0, 0]"
1,1279,refugee ng Pope Francis washed and kissed the ...,"[0, 1, 0, 0, 0, 0, 0]"
2,8330,refugee ng Many refugees do n't want to be res...,"[0, 0, 1, 0, 0, 0, 0]"
3,4063,"in-need ie ""Budding chefs , like """" Fred """" , ...","[1, 0, 0, 1, 1, 1, 0]"
4,4089,"homeless pk ""In a 90-degree view of his consti...","[1, 0, 0, 0, 0, 0, 0]"
...,...,...,...
2089,10462,"homeless gh The sad spectacle , which occurred...","[0, 0, 0, 0, 0, 0, 0]"
2090,10463,"refugee pk """""" The Pakistani police came to ou...","[0, 0, 0, 0, 0, 0, 0]"
2091,10464,"disabled ie ""When Marie O'Donoghue went lookin...","[0, 0, 0, 0, 0, 0, 0]"
2092,10465,"women lk ""Sri Lankan norms and culture inhibit...","[0, 0, 0, 0, 0, 0, 0]"


In [ ]:
tedf2.label = tedf2.label.apply(literal_eval)

In [ ]:
# Subsampling negative instances.
all_negs = trdf2[trdf2.label.apply(lambda x:sum(x) == 0)]
all_pos = trdf2[trdf2.label.apply(lambda x:sum(x) > 0)]

training_set2 = pd.concat([all_pos,all_negs[:round(len(all_pos)*0.5)]])

In [ ]:
task2_model_args = MultiLabelClassificationArgs(num_train_epochs=30,
                                                no_save=True, 
                                                no_cache=True, 
                                                overwrite_output_dir=True
                                                )
task2_model = MultiLabelClassificationModel("xlnet", 
                                            'xlnet-base-cased',
                                            num_labels=7,
                                            #pos_weight=[0.9,0.02,0.04,0.01,0.01,0.01,0.01],
                                            args = task2_model_args, 
                                            use_cuda=cuda_available)
# train model
task2_model.train_model(training_set2[['text', 'label']])
# run predictions
preds_task2, _ = task2_model.predict(tedf2.text.tolist())

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForMultiLabelSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForMultiLabelSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.weight', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream tas

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

...

Running Epoch 18 of 30:   0%|          | 0/149 [00:00<?, ?it/s]

Running Epoch 19 of 30:   0%|          | 0/149 [00:00<?, ?it/s]

Running Epoch 20 of 30:   0%|          | 0/149 [00:00<?, ?it/s]

Running Epoch 21 of 30:   0%|          | 0/149 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

## Prepare submission

In [ ]:
preds_task2, _ = task2_model.predict(tedf2.text.tolist())

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2094 [00:00<?, ?it/s]

  0%|          | 0/262 [00:00<?, ?it/s]

In [ ]:
# first, we need to create the res/ and ref/ folders, which the evaluator expects
!mkdir ref res

mkdir: cannot create directory ‘ref’: File exists
mkdir: cannot create directory ‘res’: File exists


In [ ]:
labels2file(preds_task2, 'res/task2.txt')

In [ ]:
!cat res/task2.txt | head -n 10

1,0,1,1,0,0,0
1,1,0,0,0,0,0
0,0,0,0,0,0,0
1,0,0,1,1,1,0
1,0,0,0,0,0,0
0,0,0,0,0,1,0
1,0,0,0,0,1,0
1,1,0,0,0,0,0
0,0,0,0,0,1,0
1,0,1,1,0,1,0


In [ ]:
# for subtask 1 (we convert our list of labels into a list of lists to make 
# it compatible with the labels2file function)
#labels2file(tedf1.label.apply(lambda x:[x]).tolist(), os.path.join('ref/', 'task1.txt'))
# and for subtask 2
#labels2file(tedf2.label.apply(lambda x:[x]).tolist(), os.path.join('ref/', 'task2.txt'))
labels2file(dpm.train_task2_df.label.tolist(), os.path.join('ref/', 'task2.txt'))

In [ ]:
!python3 evaluation.py . .

In [ ]:
!cat scores.txt

task2_unb:0.4799216454456416
task2_sha:0.20408163265306123
task2_pre:0.24605678233438483
task2_aut:0.2005730659025788
task2_met:0.16666666666666666
task2_com:0.39238653001464124
task2_the:0.0888888888888889
task2_avg:0.25408217312940906


# Now train the model on the full training set 2. And make predictions on unlabelled test set.

In [ ]:
test_df = pd.read_csv('task4_test.csv',sep='\t', header=None)

for i in range(0,len(test_df)):
  test_df[4][i] = test_df[2][i]+' '+test_df[3][i]+' '+test_df[4][i]

#test_df

In [ ]:
training_set2 = pd.concat([trdf2,tedf2])

# Subsampling negative instances.
all_negs = training_set2[training_set2.label.apply(lambda x:sum(x) == 0)]
all_pos = training_set2[training_set2.label.apply(lambda x:sum(x) > 0)]

training_set2 = pd.concat([all_pos,all_negs[:round(len(all_pos)*0.5)]])

task2_model_args = MultiLabelClassificationArgs(num_train_epochs=30,
                                                no_save=True, 
                                                no_cache=True, 
                                                overwrite_output_dir=True
                                                )
task2_model = MultiLabelClassificationModel("distilbert", 
                                            'distilbert-base-cased',
                                            num_labels=7,
                                            args = task2_model_args, 
                                            use_cuda=cuda_available)
# train model
task2_model.train_model(training_set2[['text', 'label']])
# run predictions
preds_task2, _ = task2_model.predict(test_df[4].tolist())

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForMultiLabelSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForMultiLabelSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight'

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1489 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


...

Running Epoch 28 of 30:   0%|          | 0/187 [00:00<?, ?it/s]

Running Epoch 29 of 30:   0%|          | 0/187 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of distilbert model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/3832 [00:00<?, ?it/s]

  0%|          | 0/479 [00:00<?, ?it/s]

In [ ]:
labels2file(preds_task2, 'res/task2.txt')
!cat res/task2.txt | head -n 10

0,0,0,0,0,0,0
0,0,1,0,0,1,0
0,0,0,0,0,0,0
0,0,0,0,0,0,0
1,0,0,0,0,0,0
1,0,0,0,0,0,0
0,0,0,0,0,1,0
0,0,0,0,0,0,0
0,0,0,0,0,0,0
0,0,0,0,0,0,0


In [ ]:
!zip submission.zip task1.txt task2.txt

  adding: task1.txt (deflated 92%)
  adding: task2.txt (deflated 97%)
